In [37]:
import pandas as pd
import csv
import numpy as np
from tqdm import tqdm
# 1. 跑過所有 rating
# 2. 比較每個 rating 跟 dot 的差距
# 3. backprobagation
# 4. 算 loss
# 5. converge 時結束（用 loss 判斷）
def matrix_factorization(R,P,I,epochs,alpha,lda):
    
    while(epochs>0):        
        epochs -= 1
        # gradient descent
        for i in tqdm(range(len(R))):
            for j in range(len(R[i])):
                if R[i][j] == 0:
                    continue
                    
                eij = np.dot(P[i],I[j].T)-R[i][j]
                for k in range(len(P[i])):
                    P[i][k] = P[i][k] - alpha*(2*I[j][k]*eij-lda*P[i][k])
                    I[j][k] = I[j][k] - alpha*(2*P[i][k]*eij-lda*I[j][k])

        if epochs%2 ==0:
            # loss calculation
            e_all = 0
            r_num = 0
            for i in range(len(R)):
                for j in range(len(R[i])):
                    if R[i][j]==0:
                        continue
                    r_num += 1
                    e_all += ((np.dot(P[i],I[j].T)).astype(int)-R[i][j])**2
            rmse = e_all/r_num
            print("epoch:",epochs,", rmse:",rmse)
            print('----------------------------------')

In [21]:
import os
import csv
data_dir = '../data'
file_folder = 'movielens'
file_name   = 'movielens_offline_train.csv'
file_path = os.path.join(data_dir,file_folder,file_name)

file_test_name   = 'movielens_offline_test.csv'
file_test_path = os.path.join(data_dir,file_folder,file_name)

In [23]:
df_train = pd.read_csv(file_path, header=None)
df_test = pd.read_csv(file_test_path, header=None)
df_train = df_train.loc[:,[0,1]]
df_test  = df_test.loc[:,[0,1]]

In [29]:
unique_users = df_train[0].unique()
unique_items = df_train[1].unique()
N = len(unique_users)
M = len(unique_items)
print(N,M)

610 9701


In [30]:
index_to_user_id = list(unique_users)
index_to_item_id = list(unique_items)
item_id_to_index = {}
user_id_to_index = {}
for _index, u_id in enumerate(unique_users):
    user_id_to_index[u_id] = _index
for _index, i_id in enumerate(unique_items):
    item_id_to_index[i_id] = _index

In [31]:
IMPLICIT = True

R = np.zeros([N,M])
for r in data:
    u_id = r[0]
    i_id = r[1]
    if IMPLICIT:
        R[user_id_to_index[u_id]][item_id_to_index[i_id]] = 1
    else:
        R[user_id_to_index[u_id]][item_id_to_index[i_id]] = r[2]

In [33]:
R.dtype

dtype('float64')

In [34]:
epochs = 100
N = len(unique_users)
M = len(unique_items)
dim = 10
P = np.random.rand(N,dim)
I = np.random.rand(M,dim)
alpha = 0.0005
lda   = 0.02

In [35]:
matrix_factorization(R,P,I,epochs,alpha,lda)

100%|██████████| 610/610 [00:08<00:00, 70.89it/s]


KeyboardInterrupt: 

In [64]:
i = 0
for i, r in df_t.iterrows():
    i+=1
    u_id = str(int(r[0]))
    i_id = str(int(r[1]))
    print(u_id,i_id)
    n = u2r[u_id]
    m = i2r[i_id]
    print(n,m)
    print("R_:",R[n][m])
    print("R':",np.dot(P[n],I.T))
    print('-----------------')

print(i)

100226


In [73]:
u_id = str(int(df_t[0][0]))
n = u2r[u_id]
rating_list = np.dot(P[n],I.T)

In [81]:
rating_list.sort()

In [84]:
np.sort(rating_list)[::-1]

array([5.84221006, 5.50993275, 5.48095541, ..., 1.59849014, 1.58650548,
       1.45987863])

# LightFM

In [5]:
import os
import numpy as np
import pandas as pd

from lightfm.datasets import fetch_stackexchange

data_dir = '../data'
file_folder = 'movielens'
file_name   = 'movielens_offline_train.csv'
file_path = os.path.join(data_dir,file_folder,file_name)

file_test_name   = 'movielens_offline_test.csv'
file_test_path = os.path.join(data_dir,file_folder,file_name)

data = fetch_stackexchange('crossvalidated',
                           test_set_fraction=0.1,
                           indicator_features=False,
                           tag_features=True)

In [9]:
train = pd.read_csv(file_path, header=None)
test = pd.read_csv(file_test_path, header=None)

In [13]:
train = train.loc[:,[0,1]]
test  = test.loc[:,[0,1]]

In [15]:
data['train']

<3221x72360 sparse matrix of type '<class 'numpy.float32'>'
	with 57830 stored elements in COOrdinate format>

In [17]:
data['train'].shape

(3221, 72360)

### build dataset

In [1]:
import os
import numpy as np
import pandas as pd
from lightfm.data import Dataset
from lightfm.datasets import fetch_stackexchange

data_dir = '../data'
file_folder = 'movielens'
file_train_name   = 'movielens_offline_train.csv'
file_train_path = os.path.join(data_dir,file_folder,file_train_name)

file_test_name   = 'movielens_offline_test.csv'
file_test_path = os.path.join(data_dir,file_folder,file_test_name)

df_train = pd.read_csv(file_train_path, header = None)


/Users/Yu/anaconda3/envs/recommender/lib/python3.6/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [70]:
dataset = Dataset()
users = df_train[0].tolist()
items = df_train[1].tolist()
data = list(zip(users,items))

In [3]:
dataset.fit((a for a in users),(a for a in items))
(interactions, weights) = dataset.build_interactions(data)

In [20]:
from lightfm import LightFM

alpha = 1e-05
epochs = 1000
num_components = 10
bpr_duration = []

model = LightFM(no_components=num_components,
                    loss='bpr',
                    learning_schedule='adagrad',
                    user_alpha=alpha,
                    item_alpha=alpha)

model.fit(interactions, epochs=epochs)

In [29]:
df_test = pd.read_csv(file_test_path, header = None)
u_items = list(set(items))

In [72]:
len(set(items))
df_test[0].tolist()[0]
np.asarray([429]*100)

9701

In [53]:
np.asarray([df_test[0].tolist()[0]]*3)

array([429, 429, 429])

In [47]:
predict(user_ids, item_ids, item_features=None, user_features=None, num_threads=1)

(9701,)

In [77]:
model.predict(np.asarray([df_test[0].tolist()[0]]),np.asarray([9700]))

array([-11.11168766])

In [63]:
np.asarray([df_test[0].tolist()[0]]*9701)

array([429, 429, 429, ..., 429, 429, 429])

In [68]:
np.asarray(u_items)

array([    1,     2,     3, ..., 32743, 65514, 98296])

In [39]:
for u in df_test[0].tolist():
    model.predict(np.asarray([df_test[0].tolist()[0]]*len(u_items)),np.asarray(u_items))
    break

ValueError: The item feature matrix specifies more features than there are estimated feature embeddings: 9701 vs 193610.

ValueError: The user feature matrix specifies more features than there are estimated feature embeddings: 610 vs 611.

In [19]:
model.predict(np.asarray([107,429,107]),np.asarray(items[0:3]))

array([-2.10124397, -6.65303564, -3.21795535])

In [17]:
users[60:63]

[107, 107, 107]

In [ ]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

model = LightFM(learning_rate=0.05, loss='bpr')
model.fit(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

In [2]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

model = LightFM(learning_rate=0.05, loss='bpr')
model.fit(R, epochs=10)

In [3]:
import helpers

In [5]:
helpers.df_to_matrix()

AttributeError: module 'helpers' has no attribute 'df_to_matrix'

In [ ]:
print(1)